In [2]:
import os
import cv2
from codecs import open as copen
from shutil import copyfile, rmtree

from sklearn.model_selection import train_test_split
from django.db.models import Q
from django.contrib.auth.models import User

from app.models import Label,Image,Batch, Comment, STATUS_CHOICES
from app.models import TODO, TAGGING, REVIEWING, DONE

In [3]:
dataset_path = os.path.join('.', 'static', 'dataset')
yolo_path = os.path.join('.', 'yolo')
yolo_anno_dir = os.path.join(yolo_path, 'annotate')
obj_path = os.path.join(yolo_anno_dir, 'obj')

if os.path.exists(obj_path):
    rmtree(obj_path)
os.makedirs(obj_path)

In [4]:
classcnt = 0
classidx = dict()
stop = ['/.', '.', './', '\.', '\.']

def get_class(label):
    global classidx, classcnt
    info = [label.brand, label.model]
#     print(info)
#     cls = "{}.{}".format(*info)
    cls = label.brand
    print
    if not any(info) or cls in stop:
        cls = "unknown"
    if not cls in classidx:
        classidx[cls] = classcnt
        classcnt += 1
    return cls

def get_coord(label):
    return label.x, label.y, label.width, label.height

def get_yolo_anno(mat, label):
    imh, imw = mat.shape[:2]
    x, y, w, h = get_coord(label)
    print(x, y, w, h)
    cls = get_class(label)
    dw = 1.0/imw
    dh = 1.0/imh
    x = (x+(x+w))/2.0
    y = (y+(y+h))/2.0
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    
#     x, y, w, h = get_coord(label)
#     rx = (x+(w/2.0))/imw
#     ry = (y+(h/2.0))/imh
#     rw = w/2.0/imw
#     rh = h/2.0/imh
#     cls = get_class(label)
    return classidx[cls], x, y, w, h

def recheck_ratio(mat, labels, filename='test.png'):
    imh, imw = mat.shape[:2]
    print(imh)
    tmp = mat.copy()
    for label in labels:
        cls, rx, ry, rw, rh = label
        w = int(rw*imw)
        h = int(rh*imh)
        x1 = int(rx*imw-w/2.0)
        y1 = int(ry*imh-h/2.0)
        x2 = int(x1+w)
        y2 = int(y1+h)
        tmp = cv2.rectangle(tmp, (x1, y1), (x2, y2), (0, 255, 0))
        w = int(rw*imw)
        h = int(rh*imh)
        x = int(rx*imw)
        y = int(ry*imh)
        tmp = cv2.circle(tmp, (x, y), h, (0,0,255))
    cv2.imwrite(filename, tmp)


In [5]:
images = Image.objects.filter(batch__status=REVIEWING)\
    .exclude(label__isnull=True)
    
for i, image in enumerate(images):
    labels = list()
    labelled =  Label.objects.filter(image=image)
    image_path = os.path.join(dataset_path, image.src_path)
    fname, ext = os.path.splitext(image.src_path)
    fname = image.id
    obj_img_path = os.path.join(obj_path, "{}.png".format(fname))
    obj_txt_path = os.path.join(obj_path, "{}.txt".format(fname))
    mat = cv2.imread(image_path)
    for label in labelled:
        ret = get_yolo_anno(mat, label)
        labels.append(ret)
        
#     !!!DEBUG
    recheck_ratio(mat, labels, "./yolo/{}.png".format(i))
    copyfile(image_path, obj_img_path)
    with open(obj_txt_path, 'w+') as fp:
        for label in labels:
            fp.write(" ".join([str(x) for x in label]))
            fp.write(os.linesep)




660 399 309 374
996 369 191 384
1080
1052 225 159 209
746 307 114 186
1080
673 561 74 103
1080
380 563 71 107
1080
1025 355 77 143
1080
970 335 126 129
756 374 162 111
1080
1084 724 80 121
1308 687 92 126
588 561 130 165
239 603 244 157
1080
492 574 66 104
1080
780 366 96 296
855 635 221 243
1080
782 377 135 270
972 455 94 196
1080
842 495 147 146
684 807 317 212
0 773 151 186
1080
1040 177 171 224
712 263 207 222
1080
546 1 277 224
1080
939 175 208 278
774 199 139 243
1080
1086 458 48 115
660 618 33 72
1080
866 490 96 180
1080
786 323 141 209
1043 361 88 189
1080
931 577 154 83
948 783 86 98
1080
806 403 183 190
1080
1166 554 152 143
1072 647 87 123
1080
947 443 182 234
1080
982 655 229 202
1080
474 622 60 80
1080
1430 207 299 450
1080
828 161 491 392
523 153 248 391
1080
844 346 101 162
1000 342 129 166
1080
99 654 182 136
1080
498 433 345 236
1080
972 462 78 134
1063 495 83 86
1080
812 319 205 349
1096 329 122 209
1080
898 269 87 159
1040 239 89 191
1080
656 617 38 72
1080
643 74 19

1272 641 101 182
1192 639 71 96
1040 749 109 160
932 763 95 146
648 765 109 152
1080
1526 667 93 148
1690 677 79 150
1098 701 85 170
1036 687 59 110
1740 521 117 142
1080
1116 231 111 150
1080
1216 569 89 136
1080
1456 599 81 136
1250 541 73 136
1080
1778 455 78 54
1292 531 93 128
562 987 156 83
1080
300 723 69 135
1080
911 625 103 127
405 657 123 172
1080
1311 652 81 100
1004 587 80 104
1080
934 891 211 178
912 739 117 124
1080
1078 731 117 148
1080
622 869 131 126
600 823 105 78
1684 793 177 176
1080
985 551 50 51
1080
1151 729 109 209
1066 715 49 115
1105 735 50 124
1633 739 115 154
396 814 106 144
1080
1548 887 218 174
425 734 95 148
1809 670 69 85
1080
1620 639 80 127
1080
481 721 109 228
1820 547 72 66
1823 595 55 93
1865 566 50 164
1080
400 729 107 154
1124 683 93 140
1080
668 707 99 138
1476 595 129 182
1298 685 91 126
1080
1334 591 91 116
1614 699 105 158
1080
796 663 73 114
1080
834 413 151 146
980 603 349 228
1080
726 657 151 224
1080
1258 693 167 288
1372 717 120 174
1500 7

310 769 143 138
1080
609 697 80 140
1041 697 59 122
1137 843 107 157
1080
1242 644 74 92
1562 857 100 162
1080
1495 703 111 135
953 663 67 102
1080
985 587 61 109
986 671 106 201
999 828 101 122
1647 695 69 95
1080
1505 577 74 119
1522 524 44 79
1080
437 772 87 119
1133 588 47 100
1080
1037 595 91 103
1080
1677 687 119 121
1425 492 43 84
1080
633 700 143 170
789 668 55 124
1080
1432 733 103 92
1080
1404 665 123 128
878 631 109 120
534 611 131 158
1080
474 811 165 212
1080
1626 857 255 194
1080
1240 801 153 258
1044 867 89 88
710 709 79 82
1080
1270 729 109 172
1538 661 165 124
1436 683 63 112
1080
450 639 107 130
1080
616 305 309 378
1078 309 339 446
1412 655 125 84
1080
1226 669 71 150
1080
602 699 135 174
1174 713 195 182
550 637 129 106
1080
1206 739 59 116
1512 603 148 132
946 549 59 78
668 815 143 184
1080
658 659 129 170
708 823 91 130
1592 781 145 168
1080
1456 587 95 108
1622 683 125 126
1080
1008 217 175 244
842 345 89 106
1330 653 147 142
1248 579 109 84
1080
658 953 167 122


910 601 79 128
1080
384 793 153 180
1080
1708 809 163 222
1080
818 777 123 158
1080
1754 701 117 138
1080
950 771 99 178
846 737 87 116
1080
618 589 143 130
334 593 212 180
1096 725 63 72
1080
1122 805 123 140
1080
678 765 239 218
1080
1596 549 279 82
1080
1026 661 97 152
1080
800 705 113 172
1624 733 99 164
1080
454 767 217 300
1080
998 611 169 118
966 457 57 94
1080
970 953 93 90
1080
898 599 139 136
1080
966 691 97 142
1298 631 131 138
1080
1038 657 125 178
1552 701 141 150
1080
384 763 67 164
1080
1062 643 101 156
1080
772 651 95 132
1080
654 573 145 108
1080
926 391 167 152
436 491 127 262
152 369 163 308
734 419 85 92
1080
1060 337 127 118
1080
556 785 121 186
1640 699 241 216
1560 561 49 100
1080
1302 739 151 224
1080
712 401 297 446
994 377 344 348
1080
466 675 109 102
1080
1056 511 159 98
1080
1328 395 168 214
1080
632 757 115 130
1108 737 115 122
1080
434 641 165 170
1080
902 973 187 100
1080
850 657 89 172
1080
958 709 93 124
1080
1288 661 107 136
1128 685 133 158
1080
474 8

1106 615 105 136
1080 687 69 72
1080
1788 667 129 140
1772 617 78 116
1308 639 79 116
1080
692 715 147 136
1458 509 105 92
1080
1121 617 58 85
1503 772 103 111
1106 727 100 134
1080
529 897 88 167
1080
1532 545 55 113
1599 638 143 234
1080
1164 695 53 96
1080
1546 767 131 124
1240 789 119 136
1080
985 457 75 90
1069 443 105 82
1080
1492 510 48 64
1080
551 710 133 180
1409 567 58 83
1080
1536 739 165 150
1080
970 687 53 74
966 827 63 46
1080
1391 593 85 109
605 747 102 91
837 713 65 93
1080
1473 559 69 87
1080
1325 710 89 125
1388 566 48 88
1080
1169 537 75 108
1735 625 119 100
1080
1176 667 113 112
628 611 103 124
1080
686 501 199 258
962 423 153 252
1080
926 839 119 110
1080
1071 873 109 125
1080
1292 731 74 59
1599 644 79 70
1080
1404 761 100 97
1080
567 603 81 123
1080
1092 837 135 166
988 657 81 74
1080
1280 821 139 152
1080
1052 589 87 120
1080
1072 705 161 142
1080
993 735 67 103
1080
1554 593 65 98
1080
809 639 50 67
781 707 89 151
1080
1140 679 71 103
1529 606 91 135
1689 543 6

...

1281 689 75 100
1080
1154 766 92 82
994 846 105 159
1073 881 88 68
719 577 47 65
730 633 72 105
1557 745 63 137
1080
1105 646 42 74
1633 644 58 78
1777 689 45 89
1381 772 54 92
1080
1482 639 52 73
1613 809 127 127
1080
914 655 97 70
1080
1308 737 93 127
1770 737 52 121
1080
917 678 36 61
890 686 44 39
781 785 73 119
1080
1263 622 79 93
841 679 104 102
1080
1511 710 93 141
1588 582 40 72
1080
1228 519 84 118
932 715 95 147
1080
1337 737 123 180
1080
1655 811 132 183
1356 767 138 143
1080
1559 686 141 240
1250 862 166 176
1125 915 171 155
1080
1711 881 137 149
1080
1652 687 95 79
1751 711 74 108
1080
1410 579 66 82
1080
1010 721 47 77
1080
617 675 72 93
636 737 96 141
1080
1050 894 74 47
1156 862 112 135
1080
1081 678 99 119
1009 595 87 127
1361 726 79 132
1180 894 151 148
1148 854 104 109
1628 751 82 94
1039 887 95 67
1080
1025 665 67 122
1239 633 79 117
1732 699 84 106
1080
1370 799 134 140
1543 667 47 87
1080
1457 665 75 117
1080
1194 918 188 87
991 831 103 143
1063 875 101 91
1080
16

1037 729 47 55
1080
1514 588 44 52
1519 651 55 87
866 705 81 73
1034 601 68 89
1080
1695 662 60 104
1080
1609 886 101 135
1080
1157 857 105 133
1080
1465 854 82 135
1605 801 76 85
1194 929 191 52
986 843 121 122
1061 867 108 91
1137 884 88 89
1080
1111 923 156 77
1239 884 159 137
1482 676 82 126
1080
1191 635 69 87
1497 699 53 83
1703 729 76 69
817 659 120 127
906 667 79 63
978 758 79 36
1080
1233 889 194 141
1194 897 113 77
1080
917 710 60 98
897 636 74 78
1080
1415 804 101 145
1306 718 60 98
1697 614 75 112
1786 612 76 124
842 681 106 108
921 662 61 63
989 745 52 49
1080
1377 681 90 101
826 643 108 133
1080
983 649 77 108
1080
1405 729 62 84
1080
1073 652 90 116
1370 694 76 111
1321 771 67 74
1080
1005 756 60 90
1130 707 65 106
1169 609 61 117
1295 803 79 130
1677 681 78 111
1080
1237 870 172 152
1080
1297 879 101 166
1085 831 119 171
1145 937 147 116
1404 855 157 99
1080
1157 820 99 164
1217 996 79 44
1080
1513 657 59 146
953 876 127 76
1080
1226 636 78 148
1260 596 42 76
1557 755 8

1214 719 76 156
1080
598 673 77 136
658 687 84 160
1080
934 797 103 162
1080
1082 735 97 140
1080
1382 797 102 144
1080
834 593 98 92
1080
1016 823 110 120
1080
490 683 183 192
550 723 88 110
1080
668 551 118 130
1080
832 677 61 74
1080
1118 663 71 106
1080
1370 735 135 148
1080
886 745 71 96
1080
1256 653 93 112
1058 703 81 106
1080
952 967 121 94
1080
1108 657 49 98
1080
1158 817 63 132
1080
620 825 73 108
1080
1246 841 69 126
1080
1434 645 77 110
1400 719 105 100
1642 737 87 114
1858 709 59 148
1080
896 701 209 212
960 763 110 130
1080
642 665 97 158
1080
1332 713 95 70
1410 703 100 186
1586 675 111 180
1080
968 915 173 158
1080
1176 589 81 92
1190 645 82 120
912 743 153 162
944 607 51 64
948 637 69 82
1648 777 75 104
1080
460 715 69 106
1080
968 935 147 142
1080
856 801 95 80
1080
1434 755 161 170
1080
764 725 91 84
1080
638 721 71 102
1080
1274 725 147 176
1080
706 765 67 88
58 697 147 152
1080
1058 617 51 98
1080
912 701 85 158
1080
1017 561 66 90
1423 777 85 111
1463 607 53 84
1

792 505 139 148
926 507 82 126
1080
814 543 120 146
574 495 131 170
1080
892 531 98 96
702 503 113 174
98 941 91 114
322 891 81 64
1080
68 797 177 140
298 829 127 98
1080
804 531 101 108
608 591 197 152
1080
4 529 274 440
1080
974 405 129 146
884 621 93 126
1080
1006 563 77 118
900 491 100 114
1080
814 649 85 94
1080
1130 593 119 238
1544 803 322 270
1080
928 611 81 86
868 645 44 96
1080
696 657 84 134
826 543 66 126
1080
780 437 111 132
976 467 87 92
1080
370 703 95 106
314 769 151 80
258 899 159 118
1080
994 459 120 136
852 459 89 126
578 575 55 64
698 557 43 64
1080
866 463 151 232
956 639 88 128
1080
654 655 105 138
792 593 69 108
1080
798 533 87 82
658 541 97 122
1080
888 479 115 130
270 653 72 70
188 619 45 96
1080
1330 901 127 88
1344 927 97 142
1080
786 535 173 84
604 533 156 146
1080
894 481 140 174
774 457 96 152
1080
862 491 253 156
1106 665 246 154
1080
832 489 201 108
626 605 179 92
1080
1034 449 142 174
874 457 105 184
1080
982 625 103 70
1080
790 509 181 172
1018 579 160

510 633 181 154
1080
730 699 136 120
1080
542 633 93 140
1080
1142 571 99 146
1080
760 561 61 124
674 603 71 102
1080
948 583 53 84
1080
284 641 313 296
1080
836 511 109 134
944 557 80 146
1080
902 373 117 218
1084 391 112 152
1080
844 491 117 100
1080
638 795 114 114
1080
1796 763 76 122
1080
960 459 73 96
858 499 115 94
1080
894 457 197 186
1080
930 635 215 304
1358 679 167 306
1080
298 871 109 114
540 819 81 74
1080
1080 511 141 192
850 557 125 94
1080
882 509 111 96
742 555 139 134
1080
1626 821 125 190
1080
638 551 109 208
840 559 89 168
1080
738 545 95 74
858 537 95 72
1080
944 505 223 156
1206 491 121 148
1080
1014 575 70 76
1080
722 547 77 58
920 635 99 108
1080
1078 479 73 188
964 475 107 186
828 503 91 66
694 487 71 88
1080
956 571 138 134
824 441 138 118
284 623 81 112
1080
858 455 161 145
1037 499 131 136
1080
860 487 109 179
1082 543 97 241
1080
1007 687 71 78
1080
500 564 39 90
1080
534 607 59 90
458 653 87 68
1080
984 467 125 132
804 539 131 112
1080
1094 513 195 176
834

In [6]:
namespath = os.path.join(yolo_anno_dir, 'obj.names')
with copen(namespath, 'w+', "utf8") as fp:    
    for k, v in sorted( classidx.items(), key=lambda x:x[1] ):
        fp.write(k)
        fp.write(os.linesep)

datapath = os.path.join(yolo_anno_dir, 'obj.data')
context = """classes= {}
train  = yolo/annotate/train.txt
valid  = yolo/annotate/test.txt
names = yolo/annotate/obj.names
backup = backup/""".format(len(classidx))
with open(datapath, 'w+') as fp:
    fp.write(context)

all_images = images.all().values_list('id')
train = all_images
test = ''
# train, test = train_test_split(all_images, test_size=0.1)
print (len(train), len(test))
with copen(os.path.join(yolo_anno_dir, 'train.txt'), 'w+', "utf8") as fp:
    for x in train:
        fname = "{}.png".format(x[0])
        fp.write(os.path.join(obj_path, fname)+os.linesep)
        
with copen(os.path.join(yolo_anno_dir, 'test.txt'), 'w+', "utf8") as fp:
    for x in test:
        fname = "{}.png".format(x[0])
        fp.write(os.path.join(obj_path, fname)+os.linesep)


3341 0


In [7]:
def create_cfg(args) :
    return """[net]
batch={}
subdivisions={}
width=608
height=608
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky


#######

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[route]
layers=-9

[convolutional]
batch_normalize=1
size=1
stride=1
pad=1
filters=64
activation=leaky

[reorg]
stride=2

[route]
layers=-1,-4

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={}
activation=linear


[region]
anchors =  0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828
bias_match=1
classes={}
coords=4
num=5
softmax=1
jitter=.3
rescore=1

object_scale=5
noobject_scale=1
class_scale=1
coord_scale=1

absolute=1
thresh = .6
random=1""".format(*args)

In [8]:
nclass = len(classidx)
train_cfg = (64, 8, (nclass+5)*5, nclass)
test_cfg = (1, 1, (nclass+5)*5, nclass)

path = os.path.join(yolo_path, 'yolo-obj-train.cfg')
with open(path, 'w+') as fp:
    fp.write(create_cfg(train_cfg))

path = os.path.join(yolo_path, 'yolo-obj-test.cfg')
with open(path, 'w+') as fp:
    fp.write(create_cfg(test_cfg))

In [9]:
# ./darknet detector train yolo/annotate/obj.data cfg/yolo-obj.cfg darknet19_448.conv.23
len(images)

3341